In [9]:
import numpy as np
def place_piece_in_center(piece_body, board_shape=(4, 4)):
    """
    将方块放置到空棋盘的中央位置
    参数:
    piece_body: 方块的body数组 (numpy array)
    board_shape: 棋盘大小，默认(20, 10)
    
    返回:
    centered_board: 包含居中方块的棋盘
    center_pos: 方块在棋盘上的中心位置 (row, col)
    """
    board_height, board_width = board_shape
    piece_height, piece_width = piece_body.shape
    # 计算中心位置
    center_row = (board_height - piece_height) // 2
    center_col = (board_width - piece_width) // 2
    # 创建空棋盘
    centered_board = np.zeros(board_shape, dtype=bool)
    # 将方块放置到中心位置
    centered_board[center_row:center_row + piece_height, center_col:center_col + piece_width] |= piece_body
    return centered_board
def expand_to_4_rotations(X):
    # X: (N, 4, 4) -> (N, 4, 4, 4)
    rotations = [np.rot90(X, k=i, axes=(1, 2)) for i in range(4)]  # 每次绕 (4,4) 平面旋转
    return np.stack(rotations, axis=1)  # (N, 4, 4, 4)

In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tetris import toolF,TetrisGame,place_piece_in_center
from tnn import TetrisNN,TetrisNN3,FrontEndNet,miniNN33
class TetrisDQN(nn.Module):
    def __init__(self, num_actions=120, dropout_rate=0.3):  # 10*4*3 = 120个可能的动作组合
        super().__init__()
        # 重用TetrisNN的特征提取部分
        self.frontend = FrontEndNet()
        self.nowpiece = miniNN33()
        self.nextpiece = miniNN33()
        
        # 重用共享层
        self.shared_fc = nn.Sequential(
            nn.Linear(128 + 32 + 32, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )
        
        # DQN输出头 - 输出每个动作的Q值
        self.q_head = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, num_actions)  # 输出所有可能动作的Q值
        )

    def forward(self, x_board, x_now, x_next):
        # 特征提取（与TetrisNN相同）
        board_features = self.frontend(x_board)    # (B, 128)
        piece_features1 = self.nowpiece(x_now)     # (B, 32)
        piece_features2 = self.nextpiece(x_next)   # (B, 32)
        
        # 特征融合
        combined_features = torch.cat([board_features, piece_features1, piece_features2], dim=1)  # (B, 192)
        shared_features = self.shared_fc(combined_features)  # (B, 128)
        
        # 输出Q值
        q_values = self.q_head(shared_features)  # (B, num_actions)
        
        return q_values
def load_pretrained_tetris_to_dqn(dqn_model, tetris_model_path):
    """
    从预训练的TetrisNN模型加载参数到DQN模型
    
    Args:
        dqn_model: TetrisDQN实例
        tetris_model_path: 预训练TetrisNN模型的路径
    """
    # 加载预训练的TetrisNN参数
    tetris_checkpoint = torch.load(tetris_model_path, map_location='cpu')
    
    # 如果保存的是完整模型状态
    if 'model_state_dict' in tetris_checkpoint:
        tetris_state_dict = tetris_checkpoint['model_state_dict']
    else:
        tetris_state_dict = tetris_checkpoint
    
    # 获取DQN模型的状态字典
    dqn_state_dict = dqn_model.state_dict()
    
    # 复制共享的参数
    shared_keys = [
        # FrontEndNet参数
        'frontend.conv1.0.weight', 'frontend.conv1.0.bias',
        'frontend.conv1.1.weight', 'frontend.conv1.1.bias',
        'frontend.conv1.1.running_mean', 'frontend.conv1.1.running_var',
        'frontend.conv2.0.weight', 'frontend.conv2.0.bias',
        'frontend.conv2.1.weight', 'frontend.conv2.1.bias',
        'frontend.conv2.1.running_mean', 'frontend.conv2.1.running_var',
        'frontend.incep1.branch1.weight', 'frontend.incep1.branch1.bias',
        'frontend.incep1.branch2.0.weight', 'frontend.incep1.branch2.0.bias',
        'frontend.incep1.branch2.2.weight', 'frontend.incep1.branch2.2.bias',
        'frontend.incep1.branch3.0.weight', 'frontend.incep1.branch3.0.bias',
        'frontend.incep1.branch3.2.weight', 'frontend.incep1.branch3.2.bias',
        'frontend.incep1.branch4.1.weight', 'frontend.incep1.branch4.1.bias',
        'frontend.bn1.weight', 'frontend.bn1.bias',
        'frontend.bn1.running_mean', 'frontend.bn1.running_var',
        'frontend.conv3.weight', 'frontend.conv3.bias',
        'frontend.conv4.weight', 'frontend.conv4.bias',
        
        # miniNN3参数
        'nowpiece.net.0.weight', 'nowpiece.net.0.bias',      # Conv2d(4,8)
        'nowpiece.net.2.weight', 'nowpiece.net.2.bias',      # Conv2d(8,16)
        'nowpiece.net.6.weight', 'nowpiece.net.6.bias',      # Linear(64,32)
        'nextpiece.net.0.weight', 'nextpiece.net.0.bias',
        'nextpiece.net.2.weight', 'nextpiece.net.2.bias',
        'nextpiece.net.6.weight', 'nextpiece.net.6.bias',
        
        # 共享FC层参数
        'shared_fc.0.weight', 'shared_fc.0.bias',
        'shared_fc.3.weight', 'shared_fc.3.bias'
    ]
    
    # 复制参数
    loaded_count = 0
    for key in shared_keys:
        if key in tetris_state_dict and key in dqn_state_dict:
            dqn_state_dict[key].copy_(tetris_state_dict[key])
            loaded_count += 1
            print(f"✓ Loaded: {key}")
        else:
            print(f"✗ Missing: {key}")
    
    print(f"\n成功加载 {loaded_count}/{len(shared_keys)} 个预训练参数")
    print("注意: Q网络输出头参数将保持随机初始化")
    
    return dqn_model


# 动作编码/解码工具函数
def encode_action(translation, rotation, slide):
    """
    将动作三元组编码为单一动作索引
    
    Args:
        translation: -4 到 5 (编码为 0-9)
        rotation: 0-3
        slide: -1,0,1 (编码为 0-2)
    
    Returns:
        action_index: 0 到 119
    """
    trans_encoded = translation + 4  # -4~5 -> 0~9
    slide_encoded = slide + 1        # -1~1 -> 0~2
    
    return trans_encoded * 12 + rotation * 3 + slide_encoded

def decode_action(action_index):
    """
    将动作索引解码为动作三元组
    
    Args:
        action_index: 0 到 119
        
    Returns:
        (translation, rotation, slide)
    """
    slide_encoded = action_index % 3
    rotation = (action_index // 3) % 4
    trans_encoded = action_index // 12
    
    translation = trans_encoded - 4  # 0~9 -> -4~5
    slide = slide_encoded - 1        # 0~2 -> -1~1
    
    return translation, rotation, slide

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random
from collections import deque, namedtuple
import matplotlib.pyplot as plt
from tqdm import tqdm
import threading
import queue
import time
import multiprocessing as mp

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"GPU内存: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# 经验回放缓冲区
Experience = namedtuple('Experience', ['state', 'action', 'reward', 'next_state', 'done'])

class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, *args):
        self.buffer.append(Experience(*args))
    
    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)
    
    def __len__(self):
        return len(self.buffer)

class PrioritizedReplayBuffer:
    """优先级经验回放"""
    def __init__(self, capacity, alpha=0.6):
        self.capacity = capacity
        self.alpha = alpha
        self.buffer = []
        self.priorities = np.zeros((capacity,), dtype=np.float32)
        self.pos = 0
        self.max_priority = 1.0
    
    def push(self, *args):
        experience = Experience(*args)
        
        if len(self.buffer) < self.capacity:
            self.buffer.append(experience)
        else:
            self.buffer[self.pos] = experience
        
        self.priorities[self.pos] = self.max_priority
        self.pos = (self.pos + 1) % self.capacity
    
    def sample(self, batch_size, beta=0.4):
        if len(self.buffer) == self.capacity:
            priorities = self.priorities
        else:
            priorities = self.priorities[:self.pos]
        
        probs = priorities ** self.alpha
        probs /= probs.sum()
        
        indices = np.random.choice(len(self.buffer), batch_size, p=probs)
        samples = [self.buffer[idx] for idx in indices]
        
        total = len(self.buffer)
        weights = (total * probs[indices]) ** (-beta)
        weights /= weights.max()
        
        return samples, indices, torch.FloatTensor(weights).to(device)
    
    def update_priorities(self, indices, priorities):
        for idx, priority in zip(indices, priorities):
            self.priorities[idx] = priority
            self.max_priority = max(self.max_priority, priority)
    
    def __len__(self):
        return len(self.buffer)

class TetrisEnvironment:
    """基于真实俄罗斯方块游戏的DQN环境"""
    def __init__(self, seed=None):
        self.seed = seed or np.random.randint(0, 10000)
        self.reset()
    
    def reset(self):
        """重置游戏环境"""
        self.game = TetrisGame(rows=20, cols=10, seed=self.seed)
        self.total_lines_cleared = 0
        self.step_count = 0
        self.max_height_history = []
        return self.get_state()
    
    def get_state(self):
        """获取当前状态 - 返回神经网络需要的三个输入"""
        # 1. 棋盘状态：20×10 -> 20×20 (左右各填充5列)
        board = self.game.board.astype(np.float32)  # (20, 10)
        
        # 左右各填充5列空白
        padded_board = np.pad(board, ((0, 0), (5, 5)), mode='constant', constant_values=0)  # (20, 20)
        
        # 转换为tensor格式 (1, 1, 20, 20)
        board_state = torch.FloatTensor(padded_board).unsqueeze(0).unsqueeze(0)
        
        # 2. 当前方块的4种旋转状态 (4, 4, 4)
        current_piece = place_piece_in_center(self.game.current.body)
        current_rotations = expand_to_4_rotations(current_piece[None])
        current_tensor = torch.FloatTensor(current_rotations.squeeze(0))
        
        # 3. 下一个方块的4种旋转状态 (4, 4, 4)
        next_piece = place_piece_in_center(self.game.next.body)
        next_rotations = expand_to_4_rotations(next_piece[None])
        next_tensor = torch.FloatTensor(next_rotations.squeeze(0))
        
        return board_state, current_tensor, next_tensor
    def step(self, action):
        """执行一个动作"""
        # 解码动作
        translation, rotation, slide = decode_action(action)
        
        # 记录执行前的状态
        old_score = self.game.score
        old_lines = getattr(self.game, 'lines_cleared', 0)
        old_height = self._get_board_height()
        
        # 执行动作
        reward = 0.0
        done = False
        
        try:
            # 1. 先执行旋转
            for _ in range(rotation % self.game.current.rotateCount):
                success = self.game.rotate()
                if not success:
                    break
            
            # 2. 然后执行水平移动
            if translation != 0:
                for _ in range(abs(translation)):
                    direction = 1 if translation > 0 else -1
                    success = self.game.move(direction)
                    if not success:
                        break
            
            # 3. 最后执行垂直移动/下落            
            if slide == 0:
                # 直接快速下落
                lines_cleared = self.game.step(4)
            else:
                # 先下落到底部
                while not self.game._collides(self.game.current, [self.game.current_pos[0] + 1, self.game.current_pos[1]]):
                    self.game.step(0)  # 向下移动一格
                # 执行滑行
                if slide == -1:
                    self.game.step(1)  # 左滑
                elif slide == 1:
                    self.game.step(2)  # 右滑
                # 强制落锁
                lines_cleared = self.game.step(4)
            
            reward += self._calculate_lock_reward(lines_cleared, old_height)
            # 检查游戏是否结束
            done = getattr(self.game, 'game_over', False)
            if done and self.step_count<40:
                # 如果游戏结束且步数少于40，给予较大惩罚
                reward -= (40-self.step_count) * 10.0
            if self.step_count>60 and done:
                reward+= 100.0
        except Exception as e:
            print(f"Error in step: {e}")
            done = True
            reward = -100.0
        
        self.step_count += 1
        next_state = self.get_state()
        
        return next_state, reward, done
    
    def _calculate_lock_reward(self, lines_cleared, old_height):
        """计算方块锁定时的奖励"""
        reward = 0.0
        
        # 1. 消除行数奖励（主要奖励）
        if lines_cleared > 0:
            line_rewards = {1: 300, 2: 700, 3: 1500, 4: 2000}
            reward += line_rewards.get(lines_cleared, lines_cleared * 200)
            self.total_lines_cleared += lines_cleared

        
        # 3. 计算layers惩罚（重点惩罚空洞层）
        board = self.game.board.astype(np.int8)
        # 计算空洞层数：上面是填充块，下面是空块的情况
        layers = ((board[:-1] == 1) & (board[1:] == 0)).sum() / 10
        # print(f"空洞层数: {layers}")
        reward -= layers*0.2  # 重点惩罚空洞层
        if layers < 0.1:
            reward += 15
        # 5. 计算special_var惩罚（高度分布不均匀性）
        height_array = np.zeros(self.game.cols)
        for col in range(self.game.cols):
            column = board[:, col]
            filled_indices = np.where(column)[0]
            if len(filled_indices) > 0:
                height_array[col] = self.game.rows - filled_indices[0]
            else:
                height_array[col] = 0
        
        # 计算special_var
        real_var = board.sum(axis=0).var()
        avg_height = np.average(height_array)
        min_height = np.min(height_array)
        
        if avg_height - min_height < 5:
            # 高度差异较小时，使用排序后的高度方差（去掉最低的一列）
            special_var = np.sort(height_array)[1:].var() if len(height_array) > 1 else 0
        else:
            # 高度差异较大时，使用真实方差
            special_var = real_var
        
        reward -= special_var * 0.3  # 惩罚高度分布不均匀
        # 6. 存活步数小奖励
        reward += 1
        return reward
    
    def _get_board_height(self):
        """获取棋盘的最大高度"""
        for row in range(self.game.rows):
            if np.any(self.game.board[row]):
                return self.game.rows - row
        return 0
    
    
    def get_info(self):
        """获取额外信息用于监控"""
        return {
            'score': self.game.score,
            'level': self.game.level,
            'lines_cleared': self.total_lines_cleared,
            'step_count': self.step_count,
            'board_height': self._get_board_height()    
        }


使用设备: cuda
GPU: NVIDIA GeForce RTX 2060
GPU内存: 12.9 GB


In [12]:

class DQNTrainer:
    def __init__(self, model, target_model, env, config):
        self.model = model.to(device)
        self.target_model = target_model.to(device)
        self.env = env
        self.config = config
        
        # 优化器
        self.optimizer = optim.Adam(self.model.parameters(), 
                                  lr=config['learning_rate'],
                                  weight_decay=config['weight_decay'])
        
        # 学习率调度器
        self.scheduler = optim.lr_scheduler.StepLR(
            self.optimizer, 
            step_size=config['lr_step_size'], 
            gamma=config['lr_gamma']
        )
        
        # 经验回放
        if config['use_prioritized_replay']:
            self.replay_buffer = PrioritizedReplayBuffer(config['buffer_size'])
        else:
            self.replay_buffer = ReplayBuffer(config['buffer_size'])
        
        # 训练统计
        self.episode_rewards = []
        self.episode_lengths = []
        self.losses = []
        self.episode_lines_cleared = []
        
        # 多线程数据生成
        self.data_queue = queue.Queue(maxsize=config['queue_size'])
        self.stop_data_generation = threading.Event()
    
    def epsilon_by_episode(self, episode):
        """epsilon衰减策略"""
        epsilon_start = self.config['epsilon_start']
        epsilon_end = self.config['epsilon_end']
        epsilon_decay = self.config['epsilon_decay']
        
        return epsilon_end + (epsilon_start - epsilon_end) * np.exp(-episode / epsilon_decay)
    
    def select_action(self, state, epsilon):
        """选择动作"""
        if random.random() > epsilon:
            with torch.no_grad():
                board, current, next_piece = state
                board = board.to(device)
                current = current.unsqueeze(0).to(device)
                next_piece = next_piece.unsqueeze(0).to(device)
                
                q_values = self.model(board, current, next_piece)
                return q_values.max(1)[1].item()
        else:
            return random.randrange(self.config['num_actions'])
    
    def data_generation_worker(self, worker_id):
        """数据生成线程"""
        local_env = TetrisEnvironment()
        
        while not self.stop_data_generation.is_set():
            try:
                # 生成一个episode的数据
                episode_data = []
                state = local_env.reset()
                epsilon = self.epsilon_by_episode(len(self.episode_rewards))
                
                for step in range(self.config['max_episode_length']):
                    action = self.select_action(state, epsilon)
                    next_state, reward, done = local_env.step(action)
                    
                    episode_data.append((state, action, reward, next_state, done))
                    
                    if done:
                        break
                    state = next_state
                # 记录该轮游戏总消行数
                lines_cleared_this_episode = local_env.get_info()['lines_cleared']
                # 将数据放入队列
                if not self.data_queue.full():
                    self.data_queue.put((worker_id, episode_data,lines_cleared_this_episode))
                    
            except Exception as e:
                print(f"Worker {worker_id} error: {e}")
                time.sleep(1)
    
    def optimize_model(self):
        """优化模型"""
        if len(self.replay_buffer) < self.config['batch_size']:
            return None
        
        # 采样批次数据
        if self.config['use_prioritized_replay']:
            experiences, indices, weights = self.replay_buffer.sample(
                self.config['batch_size'], 
                beta=min(1.0, 0.4 + len(self.episode_rewards) * 0.00001)
            )
        else:
            experiences = self.replay_buffer.sample(self.config['batch_size'])
            weights = torch.ones(self.config['batch_size']).to(device)
        
        # 解析批次数据
        batch = Experience(*zip(*experiences))
        
        # 准备状态数据
        board_batch = torch.cat([s[0] for s in batch.state]).to(device)
        current_batch = torch.stack([s[1] for s in batch.state]).to(device)
        next_batch = torch.stack([s[2] for s in batch.state]).to(device)
        
        action_batch = torch.LongTensor(batch.action).to(device)
        reward_batch = torch.FloatTensor(batch.reward).to(device)
        
        # 非终止状态掩码
        non_final_mask = torch.BoolTensor([not done for done in batch.done]).to(device)
        
        if non_final_mask.sum() > 0:
            non_final_next_board = torch.cat([s[0] for s, done in zip(batch.next_state, batch.done) if not done]).to(device)
            non_final_next_current = torch.stack([s[1] for s, done in zip(batch.next_state, batch.done) if not done]).to(device)
            non_final_next_next = torch.stack([s[2] for s, done in zip(batch.next_state, batch.done) if not done]).to(device)
        
        # 当前Q值
        current_q_values = self.model(board_batch, current_batch, next_batch).gather(1, action_batch.unsqueeze(1))
        
        # 目标Q值
        next_q_values = torch.zeros(self.config['batch_size']).to(device)
        if non_final_mask.sum() > 0:
            with torch.no_grad():
                if self.config['use_double_dqn']:
                    # Double DQN
                    next_actions = self.model(non_final_next_board, non_final_next_current, non_final_next_next).max(1)[1]
                    next_q_values[non_final_mask] = self.target_model(
                        non_final_next_board, non_final_next_current, non_final_next_next
                    ).gather(1, next_actions.unsqueeze(1)).squeeze()
                else:
                    # 标准DQN
                    next_q_values[non_final_mask] = self.target_model(
                        non_final_next_board, non_final_next_current, non_final_next_next
                    ).max(1)[0]
        
        target_q_values = reward_batch + (self.config['gamma'] * next_q_values)
        
        # 计算损失
        td_errors = target_q_values.unsqueeze(1) - current_q_values
        
        if self.config['use_huber_loss']:
            loss = F.smooth_l1_loss(current_q_values, target_q_values.unsqueeze(1), reduction='none')
        else:
            loss = F.mse_loss(current_q_values, target_q_values.unsqueeze(1), reduction='none')
        
        # 优先级权重
        loss = (loss * weights.unsqueeze(1)).mean()
        
        # 优化
        self.optimizer.zero_grad()
        loss.backward()
        
        # 梯度裁剪
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.config['grad_clip'])
        
        self.optimizer.step()
        
        # 更新优先级
        if self.config['use_prioritized_replay']:
            priorities = td_errors.abs().detach().cpu().numpy() + 1e-6
            self.replay_buffer.update_priorities(indices, priorities)
        
        return loss.item()
    
    def train(self):
        """主训练循环"""
        print("开始训练DQN...")
        
        # 启动数据生成线程
        num_workers = min(4, mp.cpu_count())
        workers = []
        for i in range(num_workers):
            worker = threading.Thread(target=self.data_generation_worker, args=(i,))
            worker.daemon = True
            worker.start()
            workers.append(worker)
        
        try:
            for episode in tqdm(range(self.config['num_episodes']), desc="Training"):
                # 从队列获取数据
                try:
                    worker_id, episode_data,lines_cleared_this_episode = self.data_queue.get(timeout=10)
                    
                    # 添加到经验回放
                    for experience in episode_data:
                        self.replay_buffer.push(*experience)
                    
                    # 记录统计信息
                    episode_reward = sum([exp[2] for exp in episode_data])
                    self.episode_rewards.append(episode_reward)
                    self.episode_lengths.append(len(episode_data))
                    self.episode_lines_cleared.append(lines_cleared_this_episode)
                except queue.Empty:
                    print("数据队列为空，跳过此次更新")
                    continue
                
                # 模型优化
                if len(self.replay_buffer) >= self.config['batch_size']:
                    for _ in range(self.config['update_frequency']):
                        loss = self.optimize_model()
                        if loss is not None:
                            self.losses.append(loss)
                
                # 更新目标网络
                if episode % self.config['target_update_frequency'] == 0:
                    self.target_model.load_state_dict(self.model.state_dict())
                
                # 学习率调度
                if episode % self.config['lr_step_size'] == 0:
                    self.scheduler.step()
                
                # 保存检查点
                if episode % self.config['save_frequency'] == 0:
                    self.save_checkpoint(episode)
                
                # 打印进度
                if episode % self.config['print_frequency'] == 0:
                    self.print_progress(episode)
        
        finally:
            # 停止数据生成线程
            self.stop_data_generation.set()
            for worker in workers:
                worker.join(timeout=5)
    
    def save_checkpoint(self, episode):
        """保存检查点"""
        checkpoint = {
            'episode': episode,
            'model_state_dict': self.model.state_dict(),
            'target_model_state_dict': self.target_model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'episode_rewards': self.episode_rewards,
            'episode_lengths': self.episode_lengths,
            'losses': self.losses,
            'config': self.config
        }
        
        torch.save(checkpoint, f'dqn_checkpoint_episode_{episode}.pth')
        
        # 保存最佳模型
        if len(self.episode_rewards) > 100:
            recent_avg = np.mean(self.episode_rewards[-100:])
            if not hasattr(self, 'best_avg_reward') or recent_avg > self.best_avg_reward:
                self.best_avg_reward = recent_avg
                torch.save(self.model.state_dict(), 'best_dqn_model.pth')
    
    def print_progress(self, episode):
        """打印训练进度"""
        if len(self.episode_rewards) == 0:
            return
            
        recent_rewards = self.episode_rewards[-100:] if len(self.episode_rewards) >= 100 else self.episode_rewards
        recent_lengths = self.episode_lengths[-100:] if len(self.episode_lengths) >= 100 else self.episode_lengths
        recent_losses = self.losses[-100:] if len(self.losses) >= 100 else self.losses
        recent_lines = self.episode_lines_cleared[-100:] if len(self.episode_lines_cleared) >= 100 else self.episode_lines_cleared
        
        avg_reward = np.mean(recent_rewards)
        avg_length = np.mean(recent_lengths)
        avg_loss = np.mean(recent_losses) if recent_losses else 0
        avg_cleared = np.mean(recent_lines) if recent_lines else 0
        epsilon = self.epsilon_by_episode(episode)
        lr = self.scheduler.get_last_lr()[0]
        
        print(f"\nEpisode {episode}")
        print(f"平均奖励: {avg_reward:.2f}")
        print(f"平均长度: {avg_length:.1f}")
        print(f"平均损失: {avg_loss:.4f}")
        print(f"平均消行: {avg_cleared:.2f}")
        print(f"Epsilon: {epsilon:.3f}")
        print(f"学习率: {lr:.6f}")
        print(f"缓冲区大小: {len(self.replay_buffer)}")

# 训练配置
config = {
    'num_episodes': 100000,
    'max_episode_length': 1000,
    'batch_size': 64,
    'buffer_size': 10000,
    'learning_rate': 5e-4,
    'weight_decay': 1e-5,
    'gamma': 0.85,
    'epsilon_start': 0.8,
    'epsilon_end': 0.05,
    'epsilon_decay': 7000,
    'target_update_frequency': 300,
    'update_frequency': 4,
    'lr_step_size': 1500,
    'lr_gamma': 0.9,
    'grad_clip': 10.0,
    'save_frequency': 2000,
    'print_frequency': 100,
    'queue_size': 50,
    'num_actions': 120,
    'use_prioritized_replay': True,
    'use_double_dqn': True,
    'use_huber_loss': True
}

# 创建训练器并开始训练
if __name__ == "__main__":
    # 创建环境
    env = TetrisEnvironment()
    dqn_model = TetrisDQN(num_actions=120, dropout_rate=0.3)
    load_pretrained_tetris_to_dqn(dqn_model, 'tetris_model3_1.2M.pth')
    # dqn_model.load_state_dict(torch.load('best_dqn_model.pth', map_location=device))#
    print("DQN模型创建完成!")
    print(f"模型参数数量: {sum(p.numel() for p in dqn_model.parameters()):,}")
    print(f"可训练参数数量: {sum(p.numel() for p in dqn_model.parameters() if p.requires_grad):,}")
    target_model = TetrisDQN(num_actions=120, dropout_rate=0.1)
    load_pretrained_tetris_to_dqn(target_model, 'tetris_model3_1.2M.pth')
    # target_model.load_state_dict(torch.load('best_dqn_model.pth', map_location=device))
    # 创建训练器
    trainer = DQNTrainer(dqn_model, target_model, env, config)
    # 开始训练
    trainer.train()
    
    print("训练完成！")

Training:  62%|██████▏   | 61902/100000 [2:40:25<1:35:00,  6.68it/s]


Episode 61900
平均奖励: -56.98
平均长度: 28.7
平均损失: 1.0620
平均消行: 0.14
Epsilon: 0.050
学习率: 0.000500
缓冲区大小: 10000


Training:  62%|██████▏   | 62002/100000 [2:40:41<2:50:14,  3.72it/s]


Episode 62000
平均奖励: 8.06
平均长度: 29.8
平均损失: 1.0798
平均消行: 0.31
Epsilon: 0.050
学习率: 0.000500
缓冲区大小: 10000


Training:  62%|██████▏   | 62102/100000 [2:40:56<1:38:47,  6.39it/s]

...


Episode 99800
平均奖励: 30.63
平均长度: 30.1
平均损失: 1.2419
平均消行: 0.37
Epsilon: 0.050
学习率: 0.000500
缓冲区大小: 10000


Training: 100%|█████████▉| 99902/100000 [4:15:44<00:14,  6.66it/s]


Episode 99900
平均奖励: 36.04
平均长度: 29.8
平均损失: 1.2176
平均消行: 0.39
Epsilon: 0.050
学习率: 0.000500
缓冲区大小: 10000


Training: 100%|██████████| 100000/100000 [4:15:59<00:00,  6.51it/s]


训练完成！
